In [2]:
# Kütüphaneleri yüklüyoruz.
import cv2
import numpy as np

In [3]:
from ultralytics import YOLO
# Burada kullanacağımız modeli seçiyoruz.
model= YOLO("yolov8l.pt") 

Burada temel mantık şu şekilde. Her bir masa için bir bölge tanımlıyoruz. Bu bölgeler masalarda bulunan kişileri kapsayacak genişlikte olmalı. Bunun için kişilerin orta noktasını kullanıyoruz. Ardından o bölgede bulunan kişilere bakıyoruz.

Burada zamanı ölçmek için şöyle bir mantık kullandım. Kişinin ilk o bölgede bulunduğu anın kaçıncı frame olduğunu kaydedip bunu güncel frame'den çıkardım. Böylece arada kaç frame fark olduğunu buldum. Sonrasında bu farkı bir frame'nin süresi ile çarptım. Bir frame'nin ne kadar sürdüğünü ise videonun FPS'sinden hesapladım. Örneğin bir video 25 FPS ise bu durumda bir frame 1/25 yani 0.04 saniye uzunluğundadır.

Normalde kişinin ilk bulunduğu anı time.time ile alıp güncel andan çıkarabiliriz. Fakat burada şöyle bir sorun var. Kodun frame'i işleme süresi videonun normal frame süresi ile uyumlu olmayabilir. Örneğin kodun buradaki işlemleri her bir frame için yapma süresi 0.1 saniye olursa bu durumda koddaki FPS 10 olacaktır. Bu durumda videodaki gerçek süre yanlış hesaplanmış olur. Bundan dolayı bu benim yaptığım şekilde hesaplamak daha sağlıklı sonuöçlar verecektir.

In [5]:
# Kullanılacak videoyu tanımladık.
kamera= cv2.VideoCapture('video.mp4')
font = cv2.FONT_HERSHEY_SIMPLEX



# Her bir masa için bölge tanımlıyoruz
region1=np.array([(100,200),(1100,200),(1100,700),(100,700)])
region1 = region1.reshape((-1,1,2))

# Videonun gerçek FPS'sini buluyoruz.
fps = kamera.get(cv2.CAP_PROP_FPS)
print(f"{fps} frames per second")

# Frame'lerin indisini bununla tutuyoruz. Yani kaçıncı frame olduğunu
count=0

# Masalardaki kişilerin kendilerine özel olan id'leri ve ilk defa kaçıncı frame'da o masada oldukları bigisini burada tutuyoruz
first_entering = {}
while True:

   
   
    ret,kare=kamera.read()
    
    # Her bir frame'da bunu arttırıyoruz
    count+=1
    if not ret:
        break
    
    imgs=cv2.cvtColor(kare,cv2.COLOR_BGR2RGB)
    
    # Resmi modele veriyoruz. Tracking yapılacağı için track modunda verdik
    results = model.track(imgs, persist=True, verbose=False)
    labels=results[0].names
    
    # Resimde kaç tane nesne varsa o kadar döner bu döngü. 
    # Yani her bir döngüde bir nesneye bakıyoruz.
    # Burada gerekli olan bilgileri alıp uygun formata çeviriyoruz.
    for i in range(len(results[0].boxes)):
        
        # Tespit edilen nesnelerin konumları.
        # x1 ve y1 sol üst köşe. x2 ve y2 sağ alt köşe koordinatları.
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        
        # Tespit edilen nesnelerin score değerleri.
        score=results[0].boxes.conf[i]
        
        # Tespit edilen nesnelerin ait hangi sınıfa ait oldukları
        label=results[0].boxes.cls[i]
        
        # Tespit edilen nesnelerin id'leri 
        ids=results[0].boxes.id[i]
        
        # Değerleri uygun bir fomata çevirdik.
        x1,y1,x2,y2,score,label,ids=int(x1),int(y1),int(x2),int(y2),float(score),int(label),int(ids)
        
        # Burada sınıf bilgisi sayılar ile tutuluyor bunlara karşılık gelen adları atadık
        name=labels[label]
        
        #  burada 0.5' lik bir threshold uyguladık. Tespit edilme değeri daha küçükse değerlendirmeye almıyoruz
        if score<0.5:
            continue
            
            
        # Sadece insan tespiti yapması için
        if name!='person':
            continue
            
        # Nesnelerin ortasının konumu buluyoruz.
        cx=int(x1/2+x2/2)
        cy=int(y1/2+y2/2)
        
        
        # Nesnenin yani kişinin orta noktasının masanın olduğu  bölgeye girip girmedğine bakıyoruz
        # Buradaki işlemleri her masa için tekrar ediyoruz.
        inside_region1=cv2.pointPolygonTest(region1,(cx,cy),False)
        
        # Eğer kişinin orta noktası masanın olduğu bölgeye girmişse if bloğunun içine girilir.
        if inside_region1>0:


            # Burada kişin id'si listenin içinde var mı diye bakıyoruz. 
            # Eğer varsa bu durumda kişi bir süredir masanın içinde demektir.


            if ids  in first_entering:
                
                # Burada kişinin ilk defa olduğu anı güncel frame değerinden çıkarıyoruz.
                fark=count-first_entering[ids]
                
                # Bu farkı bir frame süresi ile çarpıyoruz. 
                zaman=fark*(1/fps)
                zaman=int(zaman)
                # Bunu uygun formatta ekranda gösteriyoruz.
                text_zaman='Sure: '+str(zaman)+' sn'
                cv2.putText(kare,text_zaman ,(x1, y1-10), font, 0.7, (255,255,255), 2)
            
            #Yukarıdaki if bloğunun içine girmezse bu durumda kişi ilk defa masanın içinde bulunuyor demektir
            else:
                # Burada kişinin id'si ile beraber hangi an olduğunu kaydediyoruz
                first_entering[ids]=count
    
    # İsterseniz aşağıdaki satırı yorumdan kaldırıp bölgeleri ekranda gösterebilirsiniz
    #cv2.polylines(kare,[region1],True,(0,0,255),2)
    cv2.imshow("kamera",kare)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

kamera.release()
cv2.destroyAllWindows()

29.97002997002997 frames per second
